In [13]:
import numpy as np
import pandas as pd
import datetime
import sys, os

# Data parsing

In [14]:
# Parse html file
DAY = datetime.datetime.now().strftime("%d").lstrip('0')
MONTH = datetime.datetime.now().strftime("%m").lstrip('0')
# SUB = '/scrf'
SUB = ''
file_name = 'Оперативные данные _ Коронавирус COVID–19_ Официальная информация о коронавирусе в России на портале – стопкоронавирус.рф.html'

# Save of "https://xn--80aesfpebagmfblc0a.xn--p1ai/information/" page
path = f'html_data/{DAY}-{MONTH}{SUB}/{file_name}'
parse_df = pd.read_html(path, encoding='utf-8')[0]

parse_df.head()

,Регион,Выявлено,Новые,Активные ? Число больных в настоящее время,Выздоровело,Умерло
0,Москва,161397,3190,112050,47413,1934
1,Московская область,32653,846,27166,5163,324
2,Санкт-Петербург,12955,363,9718,3124,113
3,Нижегородская область,7826,211,5509,2257,60
4,Республика Дагестан,3982,127,910,3002,70


In [15]:
# Modifi data frame
parse_df = parse_df.drop(parse_df.columns[2:4] , axis=1)
parse_df.columns = ['Region/City', 'Confirmed', 'Recovered', 'Deaths']
parse_df['Date'] = datetime.datetime.now().strftime("%Y-%m-%d")
parse_df

,Region/City,Confirmed,Recovered,Deaths,Date
0,Москва,161397,47413,1934,2020-05-23
1,Московская область,32653,5163,324,2020-05-23
2,Санкт-Петербург,12955,3124,113,2020-05-23
3,Нижегородская область,7826,2257,60,2020-05-23
4,Республика Дагестан,3982,3002,70,2020-05-23
...,...,...,...,...,...
80,Севастополь,154,113,2,2020-05-23
81,Республика Алтай,92,46,0,2020-05-23
82,Сахалинская область,66,36,0,2020-05-23
83,Ненецкий автономный округ,62,10,0,2020-05-23


# Preparing parsed data

In [16]:
file_name = 'release/covid19-russia-cases-scrf.csv'
rus_df = pd.read_csv(file_name)
rus_df.tail()

,Date,Region/City,Region/City-Eng,Region_ID,Day-Confirmed,Day-Deaths,Day-Recovered,Confirmed,Deaths,Recovered
4157,2020-05-22,Ханты-Мансийский АО,Khanty-Mansiysk,86.0,80.0,1.0,18.0,1510.0,13.0,520.0
4158,2020-05-22,Челябинская область,Chelyabinsk region,74.0,80.0,0.0,31.0,1993.0,11.0,725.0
4159,2020-05-22,Чеченская Республика,Chechen Republic,95.0,20.0,0.0,8.0,1046.0,11.0,643.0
4160,2020-05-22,Ямало-Ненецкий АО,Yamalo-Nenets Autonomous Okrug,89.0,87.0,0.0,74.0,1989.0,5.0,398.0
4161,2020-05-22,Ярославская область,Yaroslavl region,76.0,103.0,0.0,24.0,2309.0,13.0,444.0


In [17]:
# Add day columns
parse_df['Day-Confirmed'] = 0
parse_df['Day-Deaths'] = 0
parse_df['Day-Recovered'] = 0

# Strip text data
rus_df['Region/City'] = rus_df['Region/City'].astype('str').str.strip('\u200b') 
parse_df['Region/City'] = parse_df['Region/City'].astype('str').str.strip('\u200b')

# Rename regions
rename_dict = {
    'Ямало-Ненецкий автономный округ' : 'Ямало-Ненецкий АО',
    'Республика Северная Осетия — Алания' : 'Республика Северная Осетия - Алания',
}

def rename(row):
    name = row['Region/City'][0]
    return pd.Series(rename_dict[name] if name in rename_dict else name)

parse_df['Region/City'] = parse_df.reset_index().groupby('index').apply(rename).reset_index(drop=True)

In [18]:
parse_df['Date'] = pd.to_datetime(parse_df['Date'])
# Minus a day if needed
# parse_df['Date'] = parse_df['Date'] - pd.Timedelta(days=1)

In [19]:
def upd(row):
    reg = row['Region/City']
    
    row['Day-Confirmed'] = row['Confirmed'] - rus_df[rus_df['Region/City'] == reg]['Day-Confirmed'].sum()
    row['Day-Deaths']    = row['Deaths'] - rus_df[rus_df['Region/City'] == reg]['Day-Deaths'].sum()
    row['Day-Recovered'] = row['Recovered'] - rus_df[rus_df['Region/City'] == reg]['Day-Recovered'].sum()
    
    row['Day-Confirmed'] = row['Confirmed'] if np.isnan(row['Day-Confirmed']) else row['Day-Confirmed']
    row['Day-Deaths']    = row['Deaths'] if np.isnan(row['Day-Deaths']) else row['Day-Deaths']
    row['Day-Recovered'] = row['Recovered'] if np.isnan(row['Day-Recovered']) else row['Day-Recovered']
    
    return row.drop('Region/City')

parse_df = parse_df.groupby('Region/City').apply(lambda df: upd(df.iloc[0])).reset_index()

In [20]:
# Check for missed regions
parse_df[np.isnan(parse_df['Day-Confirmed'])]

,Region/City,Confirmed,Recovered,Deaths,Date,Day-Confirmed,Day-Deaths,Day-Recovered


# Data checking

In [21]:
print('Russia Confirmed:', parse_df.groupby('Region/City')['Confirmed'].max().sum(),
      'Day-Confirmed:', parse_df.groupby('Region/City')['Day-Confirmed'].sum().sum())
print('Russia Deaths:', parse_df.groupby('Region/City')['Deaths'].max().sum(),
      'Day-Deaths:', parse_df.groupby('Region/City')['Day-Deaths'].sum().sum())
print('Russia Recovered:', parse_df.groupby('Region/City')['Recovered'].max().sum(),
      'Day-Recovered:', parse_df.groupby('Region/City')['Day-Recovered'].sum().sum())

# rus_regs = rus_sum.groupby('Region/City')['Recovered'].max().reset_index()
# rus_regs['Recovered-ByDay'] = rus_sum.groupby('Region/City')['Day-Recovered'].sum().reset_index(drop=True)
# rus_regs.loc[rus_regs['Recovered'] != rus_regs['Recovered-ByDay']]
# # rus_regs.to_csv('rus_regs.csv')

Russia Confirmed: 335882 Day-Confirmed: 9434.0
Russia Deaths: 3388 Day-Deaths: 139.0
Russia Recovered: 107936 Day-Recovered: 8111.0


# Data saving

In [22]:
# Save full update table
parse_df.to_csv(f'covid19-russia-cases-upd-full-scrf.csv', index=False)

In [23]:
# Drop rows without day changes
parse_df = parse_df[(parse_df['Day-Confirmed'] != 0) |
                (parse_df['Day-Deaths'] != 0) |
                (parse_df['Day-Recovered'] != 0)]

In [24]:
parse_df.to_csv(f'old_data/covid19-russia-cases-upd{DAY}-{MONTH}-scrf.csv', index=False)
parse_df.to_csv('covid19-russia-cases-upd-scrf.csv', index=False)